# 🧬 Introduction to Strands Agents: Building a HealthOmics Workflow Orchestrator

This notebook provides a comprehensive introduction to the Strands Agents framework by building a real-world AWS HealthOmics Workflow Orchestrator Agent from scratch.

## 🎯 What You'll Learn

1. **Strands Agents Fundamentals** - Core concepts and architecture
2. **MCP Integration** - Model Context Protocol for tool connectivity
3. **Agent Construction** - Step-by-step agent building process
4. **Tool Integration** - Adding custom tools and MCP servers

## 🏗️ What We'll Build

A **HealthOmics Workflow Orchestrator Agent** that can:
- Generate and manage HealthOmics workflows
- Start and monitor workflow runs
- Analyze workflow performance and optimize resource usage
- Troubleshoot failed workflows with detailed diagnostics
- Validate workflow definitions (WDL/CWL)

---

## 📚 Part 1: Strands Agents Fundamentals

### What are Strands Agents?

Strands Agents is a Python framework for building AI agents that can:
- Use tools to interact with external systems
- Maintain conversation context and memory
- Work with multiple LLM providers (Bedrock, OpenAI, etc.)
- Integrate with protocols like MCP (Model Context Protocol)
- Deploy to various environments (standalone, container, Bedrock AgentCore, etc.)

### Key Components

1. **Agent** - The core AI assistant with a specific role and capabilities
2. **Model** - The underlying LLM (Claude, GPT, etc.)
3. **Tools** - Functions the agent can call to perform actions
4. **System Prompt** - Instructions that define the agent's behavior
5. **MCP Integration** - Protocol for connecting to external tool servers

Let's start building!

## 🛠️ Step 1: Environment Setup

First, let's install the required dependencies and set up our environment.

In [ ]:
# Install required dependencies
!pip install pandas --upgrade --quiet
!pip install strands-agents boto3 uv --quiet

# Set up environment for tool consent bypass (for automated workflows)
import os
os.environ["BYPASS_TOOL_CONSENT"] = "true"

print("✅ Environment setup complete!")

## 🧱 Step 2: Basic Agent Creation

Let's start with the simplest possible Strands agent to understand the core concepts.

In [ ]:
from strands import Agent
from strands.models import BedrockModel

# Create a basic agent with no tools
basic_agent = Agent(
    name="Basic HealthOmics Assistant",
    description="A simple assistant for HealthOmics questions",
    # Define the model the agent will use
    model=BedrockModel(
        # model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
        model_id="global.anthropic.claude-sonnet-4-20250514-v1:0",
        temperature=0.1
    )
)

# Test the basic agent
response = basic_agent("What is AWS HealthOmics?")
print("🤖 Basic Agent Response:")
print(response)
print(f"\n📊 Tokens used: {response.metrics.accumulated_usage.get('totalTokens', 0)}")

## 🔧 Step 3: Adding Custom Tools

Now let's add some custom tools to make our agent more capable. We'll start with a simple tool for workflow monitoring. The agent can call this tool when a workflow starts to wait on it for a specified time. When the workflow reaches a terminal state or the wait time is exceeded the tool will return information about the status. Agents autonomously select when to run a tool based on the signature and description of the tool. The agent will determine what arguments to send to the tool and what to do with the response.

In [ ]:
import time
import boto3
from botocore.exceptions import ClientError
from strands import tool

@tool
def wait_for_workflow(run_id: str, max_wait_minutes: int = 60) -> str:
    """
    Wait for a HealthOmics workflow run to complete by polling the run status.
    Checks the run status every 30 seconds and returns early when the run reaches a terminal state.
    
    Args:
        run_id: The HealthOmics run ID to monitor
        max_wait_minutes: Maximum time to wait in minutes (default: 60, max: 120 for safety)
    
    Returns:
        str: Status message indicating the final run state and duration
    """
    # Limit maximum wait time for safety
    max_wait_minutes = min(max(max_wait_minutes, 1), 120)
    
    print(f"🔍 Monitoring HealthOmics run {run_id} (max wait: {max_wait_minutes} minutes)...")
    

    omics_client = boto3.client('omics', region_name='us-east-1')
    return omics_client.get_run(id=run_id)



Now that our tool is defined, lets create an agent that can help us monitor the status of a HealthOmics workflow run using the tool we created above. 

In [ ]:
# Create an agent with our custom tool
agent_with_tools = Agent(
    name="HealthOmics Assistant with Tools",
    description="A HealthOmics assistant that can monitor workflow runs",
    model=BedrockModel(
        # model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
        model_id="global.anthropic.claude-sonnet-4-20250514-v1:0",
        temperature=0.1
    ),
    tools=[wait_for_workflow],
    system_prompt="""
    You are a helpful AWS HealthOmics assistant. When users ask about monitoring workflows,
    use the wait_for_workflow tool to check run status. Always provide clear, actionable guidance.
    """
)



Lets find out the latest HealthOmics workflow run that we can use to test

In [ ]:
omics_client = boto3.client('omics')
latest_run_id = omics_client.list_runs()['items'][0]['id']
print(latest_run_id)

Now that we have an example workflow run ID, lets test the newly created agent with an example prompt

In [ ]:
# Test the agent with tools
response = agent_with_tools(f"Can you monitor workflow run {latest_run_id} for me?")
print("🤖 Agent with Tools Response:")
print(response)

# Verify which tool was used
print(f"\n📊 Tools used: {list(response.metrics.tool_metrics.keys())}")

## 🌐 Step 4: MCP Integration - Connecting to External Tool Servers

Now let's integrate with a tool set provided by an MCP (Model Context Protocol) server connected to the AWS HealthOmics MCP server. First, we create the client which fetches the MCP server from and runs it using `uv`. MCP servers can be remote or local and can communicate on different protocols. For simplicity (and lower latency) we deploy the agent locally and managed by the agent communicating via STDIO.

In [ ]:
from strands.tools.mcp import MCPClient
from mcp import stdio_client, StdioServerParameters
import logging

# Configure logging to see what's happening
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def create_mcp_client():
    """Create an mcp client for the HealthOmics server"""

    session = boto3.Session()
    credentials = session.get_credentials()
    
    # Prepare environment variables with AWS credentials
    # This is only required in notebooks where the host metadata endpoint is blocked
    env_vars = {
        "AWS_REGION": session.region_name or 'us-east-1',
    }
    
    # Setup MCP client for AWS HealthOmics
    mcp_client = MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="uvx",
                args=["awslabs.aws-healthomics-mcp-server@latest"],
                env=env_vars
            ),
        )
    )

    return mcp_client

# Create MCP client
print("🔌 Setting up MCP connection to HealthOmics server...")
mcp_client = create_mcp_client()
print("✅ MCP client created!")

Now, lets inspect the MCP client to see what tools the server has made available. We will also create an agent and ask it some questions that will require it to use the MCP tools to answer.

In [ ]:
# Use the MCP client to discover available tools and create a persistent agent
with mcp_client:
    # Get the tools from the MCP server
    mcp_tools = mcp_client.list_tools_sync()
    print(f"🛠️ Found {len(mcp_tools)} MCP tools from HealthOmics server:")
    
    # Show first few tools
    for i, tool in enumerate(mcp_tools[:5]):
        print(f"  {i+1}. {tool.tool_name}")
    
    if len(mcp_tools) > 5:
        print(f"  ... and {len(mcp_tools) - 5} more tools")

# Store tools and MCP client for reuse in later cells
healthomics_mcp_tools = mcp_tools
healthomics_mcp_client = mcp_client

# Create agent with both custom tools and MCP tools (available for later use)
healthomics_agent = Agent(
    name="HealthOmics Workflow Orchestrator",
    description="""
    I'm your AWS HealthOmics workflow orchestrator. I can help you:
    
    - Create and manage HealthOmics workflows
    - Start and monitor workflow runs with real-time status monitoring
    - Analyze workflow performance and optimize resource usage
    - Troubleshoot failed workflows
    - Lint and validate workflow definitions (WDL/CWL)
    - Package workflows for deployment
    
    I have access to comprehensive HealthOmics APIs and can guide you through
    genomics workflow management on AWS.
    """,
    model=BedrockModel(
        # model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
        model_id="global.anthropic.claude-sonnet-4-20250514-v1:0",
        temperature=0.1
    ),
    tools=healthomics_mcp_tools + [wait_for_workflow],  # Combine MCP tools with custom tools
    system_prompt="""
    You are an expert AWS HealthOmics workflow orchestrator agent. Your role is to:
    
    1. Help users create, deploy, and manage HealthOmics workflows
    2. Monitor workflow execution and provide real-time status updates
    3. Analyze workflow performance and suggest optimizations
    4. Troubleshoot workflow failures with detailed diagnostics
    5. Validate workflow definitions and ensure best practices
    
    Always provide clear, actionable guidance and use the HealthOmics MCP tools
    to perform operations. Be proactive in suggesting optimizations and best practices.

    When creating workflows, follow these guidelines:
    1. Unless otherwise instructed, create workflows in WDL 1.1
    2. Ensure all tasks have suitable cpu, memory and container directives favoring use of containers from quay.io
    3. Use the LintAHOWorkflowDefinition or LintAHOWorkflowBundle tools to ensure correctness
    4. When possible and logical, scatter over inputs or genomic intervals to improve computational efficiency
    5. When creating updates to existing workflows, create workflow versions rather than new workflows
    6. Use `set -euo pipefail` in task commands
    7. `echo` the names and values of task inputs in the task command to assist with debugging
    
    IMPORTANT: When you start a workflow run, you will receive a run ID. Use the wait_for_workflow tool 
    with this run ID to monitor the run status until completion. This tool polls the HealthOmics API 
    every 30 seconds and returns early when the run reaches a terminal state.
    """
)

print("\n🤖 Testing the full agent with MCP tools...")

# Test the agent with a real HealthOmics query using the MCP client context
with healthomics_mcp_client:
    response = healthomics_agent("What regions is HealthOmics available in and what workflows are available in my account?")
    print("\n📋 Agent Response:")
    print(response)
    
    print(f"\n📊 Execution Metrics:")
    print(f"  • Total tokens: {response.metrics.accumulated_usage.get('totalTokens', 0)}")
    print(f"  • Execution time: {sum(response.metrics.cycle_durations):.2f} seconds")
    print(f"  • Tools used: {list(response.metrics.tool_metrics.keys())}")

print("\n✅ HealthOmics agent is now available for experimentation in later cells!")
print("   Use 'healthomics_agent' variable to interact with the agent")
print("   Remember to use 'with healthomics_mcp_client:' context for MCP tool access")

## 🧪 Step 5: Experiment with Your Agent

Now that we have a fully functional HealthOmics agent with MCP tools, let's experiment with different prompts to see what it can do! The agent is available as `healthomics_agent` and has access to all HealthOmics MCP tools.

### 💡 **Suggested Prompts to Try:**
- "Find a workflow in my account related to somatic variant calling, show me the details of the workflow."
- "Create a simple hello-world workflow in WDL"
- "List all workflow runs from the past week"
- "Analyze run <insert run id here> and suggest optimizations"
- "What are the best practices for genomics workflows in HealthOmics?"
- "Have any runs from the last week failed, if they have, pick one a provide a diagnosis"

### 🔧 **How to Use:**
1. Modify the prompt in the cell below
2. Run the cell to see the agent's response
3. Try different prompts to explore the agent's capabilities
4. The agent will automatically use the appropriate MCP tools to answer your questions

In [ ]:
# 🧪 EXPERIMENT CELL - Try different prompts here!
# Change the prompt below to experiment with different questions

experiment_prompt = "Find a workflow in my account related to somatic variant calling, show me the details of the workflow."

print(f"🔍 Asking agent: {experiment_prompt}")
print("=" * 60)

# Use the agent with MCP client context
with healthomics_mcp_client:
    response = healthomics_agent(experiment_prompt)
    print(response)
    
    print(f"\n📊 Execution Metrics:")
    print(f"  • Total tokens: {response.metrics.accumulated_usage.get('totalTokens', 0)}")
    print(f"  • Execution time: {sum(response.metrics.cycle_durations):.2f} seconds")
    print(f"  • Tools used: {list(response.metrics.tool_metrics.keys())}")

In [ ]:
# 🧪 EXPERIMENT CELL 2 - Try another prompt!
# This cell is ready for your next experiment

experiment_prompt_2 = """
  Create a simple WDL workflow that says hello to genomics researchers and display it in your response.
  Check that the workflow syntax is correct, then create it in HealthOmics.
  """

print(f"🔍 Asking agent: {experiment_prompt_2}")
print("=" * 60)

with healthomics_mcp_client:
    response = healthomics_agent(experiment_prompt_2)
    print(response)
    
    print(f"\n📊 Execution Metrics:")
    print(f"  • Total tokens: {response.metrics.accumulated_usage.get('totalTokens', 0)}")
    print(f"  • Execution time: {sum(response.metrics.cycle_durations):.2f} seconds")
    print(f"  • Tools used: {list(response.metrics.tool_metrics.keys())}")

In [ ]:
# 🧪 EXPERIMENT CELL 3 - Add your own prompt!
# Copy this cell or modify it to try more experiments

# TODO: Replace this with your own prompt
your_prompt = "How can I optimize genomics-ai-workshop-mutect2 workflow"

print(f"🔍 Your experiment: {your_prompt}")
print("=" * 60)

with healthomics_mcp_client:
    response = healthomics_agent(your_prompt)
    print(response)
    
    print(f"\n📊 Execution Metrics:")
    print(f"  • Total tokens: {response.metrics.accumulated_usage.get('totalTokens', 0)}")
    print(f"  • Execution time: {sum(response.metrics.cycle_durations):.2f} seconds")
    print(f"  • Tools used: {list(response.metrics.tool_metrics.keys())}")

### 🎯 **What You Can Explore:**

The agent has access to these MCP tools and can help you with:

**Workflow Management:**
- List, create, and get details about workflows
- Create workflow versions
- Validate and lint workflow definitions

**Workflow Execution:**
- Start workflow runs
- Monitor run status and progress
- Get run details and logs
- Cancel running workflows

**Data Management:**
- Work with reference stores and sequence stores
- Import and export genomics data
- Manage annotation stores

**System Information:**
- Check supported regions
- Get service quotas and limits
- Understand HealthOmics capabilities

Feel free to ask complex questions - the agent will use multiple tools if needed to provide comprehensive answers!

## 🎓 Summary: What We've Built

Congratulations! You've successfully built a production-ready **HealthOmics Workflow Orchestrator Agent** using Strands Agents. Here's what we accomplished:

### 🏗️ **Architecture Components**
1. **Core Agent** - Built with Strands framework and Bedrock Claude model
2. **Custom Tools** - Added workflow monitoring capabilities
3. **MCP Integration** - Connected to AWS HealthOmics MCP server for comprehensive tools
4. **Interactive Experimentation** - Ready-to-use cells for testing different prompts

### 🛠️ **Key Features**
- **Workflow Management**: Create, deploy, and manage HealthOmics workflows
- **Real-time Monitoring**: Track workflow runs with automatic status polling
- **Performance Analysis**: Analyze resource usage and suggest optimizations
- **Failure Diagnostics**: Troubleshoot issues with detailed error reporting
- **Validation**: Lint and validate WDL/CWL workflow definitions
- **Best Practices**: Built-in genomics workflow guidance from the system prompt
- **Interactive Learning**: Experiment with different prompts to explore capabilities
